In [ ]:
import os
from pathlib import Path

import pefile
import polars as pl
from dotenv import load_dotenv


# Paths

In [ ]:
# imports csv
base = Path("../game-dump/dumps")
dump_imports_p = base / "dump-imports.csv"
old_iat_p = base / "old-iat.csv"
byte_calls_p = base / "broken-byte-calls.csv"
inst_calls_p = base / "broken-analyzed-calls.csv"

In [ ]:
names_map_p = "../game-dump/names-map.csv"
names_map = pl.read_csv(names_map_p)

In [ ]:
load_dotenv(Path(os.path.abspath('')).parent / ".env")
base_to_exe = Path(os.getenv("BASE_TO_EXE", "./"))
original_dump_path = base_to_exe / "GAME_dump.exe"
patched_path = base_to_exe / "GAME_patched.exe"

# Parsing import table

In [ ]:
dump_imports = pl.read_csv(dump_imports_p).filter(pl.col("Module") != "game.exe")

In [ ]:
dump_imports = dump_imports.with_columns(pl.col("Function").str.replace_all(" ", ""))
names_map = names_map.with_columns(
    pl.col("undecorated").str.replace_all(" ", "").alias("Function")
)

dump_imports = dump_imports.join(names_map.drop("undecorated"), on="Function", how="left").with_columns(
    pl.coalesce("decorated", "Function").alias("Function")
).drop("decorated")

In [ ]:
dump_imports = dump_imports.with_columns(("0x" + pl.col("Address").str.to_lowercase()).alias("Address"))
dump_imports = dump_imports.unique("Address", keep='first')

In [ ]:
dump_imports.head()

# Gathering imports from old IAT

In [ ]:
iat = pl.read_csv(old_iat_p)

iat = iat.rename({"Address": "Calladdr", "Destination": "Address"})
iat = iat.with_columns(("0x" + pl.col("Address").str.to_lowercase()).alias("Address"))

# iat = iat.with_columns((pl.col("Address").shift(+1).str.slice(2).str.to_integer(base=16) == 0).alias("first").fill_null(True))
# iat = iat.with_columns((pl.col("Address").shift(-1).str.slice(2).str.to_integer(base=16) == 0).alias("last").fill_null(True))

iat = iat.join(dump_imports, on='Address', how='left')

In [ ]:
iat2 = iat.filter(pl.col("Function").is_null())
iat2 = iat2.filter(pl.col("Address").str.slice(2).str.to_integer(base=16) != 0)
iat2

<0x023e3673> points to an intermediate call which i believe, is an obfuscated jump to user32.dll!wsprintfA

Cancel forwarding imports (e.g. kernel32.dll,AllocateHeap -> ntdll.dll,RtlReAllocateHeap)

In [ ]:
systemroot = "C:/Windows/System32/"
forwarding_modules = ["kernel32.dll", "user32.dll"]

unforward_map: dict[str, tuple[str, str]] = dict()

forwarded = iat.filter(pl.col("Module") == "ntdll.dll")

for modname in forwarding_modules:
    modpath = systemroot + modname
    number = 0

    dll = pefile.PE(modpath)
    dll.full_load()
    for exp in dll.DIRECTORY_ENTRY_EXPORT.symbols:
        name = exp.name.decode() if exp.name else f"Ordinal#{exp.ordinal}"
        forward_to = ''
        if exp.forwarder:
            forward_to = exp.forwarder.decode().removeprefix("NTDLL.")
        if forwarded.filter(pl.col("Function") == forward_to).shape[0] > 0:
            number += 1
            unforward_map[forward_to] = (modname, name)
    
    print(f'For {modname} there are {number} forwards')

In [ ]:
for func in iat.filter(pl.col("Module") == "ntdll.dll")["Function"]:
    if func not in unforward_map:
        print(f"Func {func} from ntdll.dll is not found in forward map")
        continue

    origmod, origfunc = unforward_map[func]
    condition = (pl.col("Module") == "ntdll.dll") & (pl.col("Function") == func)
    iat = iat.with_columns(
        pl.when(condition).then(pl.lit(origmod)).otherwise("Module").alias("Module")
    ).with_columns(pl.when(condition).then(pl.lit(origfunc)).otherwise("Function").alias("Function"))

In [ ]:
iat.write_csv(old_iat_p + '2.csv')

In [ ]:
def valid(df: pl.DataFrame) -> bool:
    if df.shape[0] == 0:
        return False
    return df['Module'][0] != ''

In [ ]:
iat_seg = iat.fill_null('').with_columns(
    (pl.col("Module") != pl.col("Module").shift(1)).cum_sum().alias("segment_id")
).fill_null(0)
segments = [
    group.drop("segment_id") for _, group in iat_seg.group_by("segment_id", maintain_order=True) if valid(group)
]


# Constructing new IDT

In [ ]:
pass

# Fix thunks

# Fix calls

# Patch PE